# Nanodegree Engenheiro de Machine Learning
## Aprendizagem Supervisionada
## Projeto 2: Construindo um Sistema de Intervenção para Estudantes

Bem-vindo ao segundo projeto do Nanodegree de Machine Learning! Neste Notebook, alguns templates de código já foram fornecidos, e será o seu trabalho implementar funcionalidades necessárias para completar este projeto com êxito. Seções que começam com **'Implementação'** no cabeçalho indicam que o bloco de código que se segue precisará de funcionalidades adicionais que você deve fornecer. Instruções serão providenciadas para cada seção e as especificações para cada implementação estarão marcadas no bloco de código com o comando `'TODO'`. Tenha certeza de ler atentamente todas as instruções!

Além do código implementado, haverá questões relacionadas ao projeto e à implementação que você deve responder. Cada seção em que você tem que responder uma questão será antecedida de um cabeçalho **'Questão X'**. Leia atentamente cada questão e escreva respostas completas nas caixas de texto subsequentes que começam com **'Resposta: '**. O projeto enviado será avaliado baseado nas respostas para cada questão e a implementação que você forneceu.  

>**Nota:** Células de código e Markdown podem ser executadas utilizando o atalho de teclado **Shift + Enter**. Além disso, as células Markdown podem ser editadas, um clique duplo na célula entra no modo de edição.

### Questão 1 - Classificação versus Regressão
*Seu objetivo neste projeto é identificar estudantes que possam precisar de intervenção antecipada antes de serem reprovados. Que tipo de problema de aprendizagem supervisionada é esse: classificação ou regressão? Por quê?*

**Resposta: **
Em problemas aprendizagem supervisionada de classificação o conjunto de dados é limitado e discreto. Em problemas de aprendizagem supervisionada de Regressão o conjunto de dados é contínuo. No problema em questão temos conjuntos de dados discretos ou que poderão ser convertidos em dados discretos para teste e treinamento. Sendo assim este problema é um tipo de aprendizagem supervisionada de Classificação. 

## Observando os Dados
Execute a célula de código abaixo para carregar as bibliotecas de Python necessárias e os dados sobre os estudantes. Note que a última coluna desse conjunto de dados, `'passed'`, será nosso rótulo alvo (se o aluno foi ou não aprovado). As outras colunas são atributos sobre cada aluno.

In [1]:
# Importar bibliotecas
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Ler os dados dos estudantes
student_data = pd.read_csv("student-data.csv")
print "Os dados dos estudantes foram lidos com êxito!"

Os dados dos estudantes foram lidos com êxito!


### Implementação: Observando os Dados
Vamos começar observando o conjunto de dados para determinar quantos são os estudantes sobre os quais temos informações e entender a taxa de graduação entre esses estudantes. Na célula de código abaixo, você vai precisar calcular o seguinte:
- O número total de estudantes, `n_students`.
- O número total de atributos para cada estudante, `n_features`.
- O número de estudantes aprovados, `n_passed`.
- O número de estudantes reprovados, `n_failed`.
- A taxa de graduação da classe, `grad_rate`, em porcentagem (%).


In [2]:
# TODO: Calcule o número de estudante
n_students = len(student_data)

# TODO: Calcule o número de atributos
n_features = len(student_data.columns[:-1])

# TODO: Calcule o número de alunos aprovados
n_passed = len(student_data[student_data["passed"] == "yes"])

# TODO: Calcule o número de alunos reprovados
n_failed = len(student_data[student_data["passed"] == "no"])

# TODO: Calcule a taxa de graduação
grad_rate = (float(n_passed)/n_students)*100

# Imprima os resultados
print "Número total de estudantes: {}".format(n_students)
print "Número de atributos: {}".format(n_features)
print "Número de estudantes aprovados: {}".format(n_passed)
print "Número de estudantes reprovados: {}".format(n_failed)
print "Taxa de graduação: {:.2f}%".format(grad_rate)

Número total de estudantes: 395
Número de atributos: 30
Número de estudantes aprovados: 265
Número de estudantes reprovados: 130
Taxa de graduação: 67.09%


## Preparando os Dados
Nesta seção, vamos preparara os dados para modelagem, treinamento e teste.

### Identificar atributos e variáveis-alvo
É comum que os dados que você obteve contenham atributos não numéricos. Isso pode ser um problema, dado que a maioria dos algoritmos de machine learning esperam dados númericos para operar cálculos.

Execute a célula de código abaixo para separar os dados dos estudantes em atributos e variáveis-alvo e verificar se algum desses atributos é não numérico.

In [3]:
# Extraia as colunas dos atributo
feature_cols = list(student_data.columns[:-1])

# Extraia a coluna-alvo 'passed'
target_col = student_data.columns[-1] 

# Mostre a lista de colunas
print "Colunas de atributos:\n{}".format(feature_cols)
print "\nColuna-alvo: {}".format(target_col)

# Separe os dados em atributos e variáveis-alvo (X_all e y_all, respectivamente)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Mostre os atributos imprimindo as cinco primeiras linhas
print "\nFeature values:"
print X_all.head()

Colunas de atributos:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Coluna-alvo: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Pré-processar Colunas de Atributo

Como você pode ver, há muitas colunas não numéricas que precisam ser convertidas! Muitas delas são simplesmente `yes`/`no`, por exemplo, a coluna `internet`. É razoável converter essas variáveis em valores (binários) `1`/`0`.

Outras colunas, como `Mjob` e `Fjob`, têm mais do que dois valores e são conhecidas como variáveis categóricas. A maneira recomendada de lidar com esse tipo de coluna é criar uma quantidade de colunas proporcional aos possíveis valores (por exemplo, `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc), e assinalar `1` para um deles e `0` para todos os outros.

Essas colunas geradas são por vezes chamadas de _variáveis postiças_ (_dummy variables_), e nós iremos utilizar a função [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) para fazer essa conversão. Execute a célula de código abaixo para executar a rotina de pré-processamento discutida nesta seção.

In [4]:
def preprocess_features(X):
    ''' Pré-processa os dados dos estudantes e converte as variáveis binárias não numéricas em
        variáveis binárias (0/1). Converte variáveis categóricas em variáveis postiças. '''
    
    # Inicialize nova saída DataFrame
    output = pd.DataFrame(index = X.index)

    # Observe os dados em cada coluna de atributos 
    for col, col_data in X.iteritems():
        
        # Se o tipo de dado for não numérico, substitua todos os valores yes/no por 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # Se o tipo de dado for categórico, converta-o para uma variável dummy
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Reúna as colunas revisadas
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementação: Divisão dos Dados de Treinamento e Teste
Até agora, nós convertemos todos os atributos _categóricos_ em valores numéricos. Para o próximo passo, vamos dividir os dados (tanto atributos como os rótulos correspondentes) em conjuntos de treinamento e teste. Na célula de código abaixo, você irá precisar implementar o seguinte:
- Embaralhe aleatoriamente os dados (`X_all`, `y_all`) em subconjuntos de treinamento e teste.
  - Utilizar 300 pontos de treinamento (aproxidamente 75%) e 95 pontos de teste (aproximadamente 25%).
  - Estabelecer um `random_state` para as funções que você utiliza, se a opção existir.
  - Armazene os resultados em `X_train`, `X_test`, `y_train` e `y_test`.

In [5]:
# TODO: Importe qualquer funcionalidade adicional de que você possa precisar aqui
from sklearn.model_selection import train_test_split 
# TODO: Estabeleça o número de pontos de treinamento
num_train = 300

# Estabeleça o número de pontos de teste
num_test = X_all.shape[0] - num_train

# TODO: Emabaralhe e distribua o conjunto de dados de acordo com o número de pontos de treinamento e teste abaixo
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=42)

# Mostre o resultado da distribuição
print "O conjunto de treinamento tem {} amostras.".format(X_train.shape[0])
print "O conjunto de teste tem {} amostras.".format(X_test.shape[0])

O conjunto de treinamento tem 300 amostras.
O conjunto de teste tem 95 amostras.


## Treinando e Avaliando Modelos
Nesta seção, você irá escolher 3 modelos de aprendizagem supervisionada que sejam apropriados para esse problema e que estejam disponíveis no `scikit-learn`. Primeiro você irá discutir o raciocínio por trás da escolha desses três modelos considerando suas vantagens e desvantagens e o que você sabe sobre os dados. Depois você irá ajustar o modelo a diferentes tamanhos de conjuntos de treinamento (com 100, 200 e 300 pontos) e medir a pontuação F<sub>1</sub>. Você vai precisar preencher três tabelas (uma para cada modelo) que mostrem o tamanho do conjunto de treinamento, o tempo de treinamento, o tempo de previsão e a pontuação F<sub>1</sub> no conjunto de treinamento.

**Os seguintes modelos de aprendizagem supervisionada estão atualmente disponíveis no **[`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html)** para você escolher:**
- Gaussian Naive Bayes (GaussianNB)
- Árvores de Decisão
- Métodos de agregação (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Método do gradiente estocástico (SGDC)
- Máquinas de vetores de suporte (SVM)
- Regressão logística

### Questão 2 - Aplicação dos Modelos
*Liste três modelos de aprendizagem supervisionada que são apropriadas para esse problema. Para cada modelo escolhido:*
- Descreva uma aplicação em mundo real na indústria em que o modelo pode ser aplicado. *(Talvez você precise fazer um pouco de pesquisa para responder essa questão – dê as devidas referências!)* 
- Quais são as vantagens do modelo; quando ele tem desempenho melhor? 
- Quais são as desvantagens do modelo, quando ele tem desempenho pior?
- O que faz desse modelo um bom candidato para o problema, considerando o que você sabe sobre os dados?

**Resposta: **
### Gaussian Naive Bayes (GaussianNB) (Candidato): 
- **Aplicações:**
    - Classificação de textos/Filtragem de spam/Análise de sentimento: classificadores Naive Bayes utilizados principalmente em classificação de textos (devido a um melhor resultado em problemas de classes múltiplas e regra de independência) têm maior taxa de sucesso em comparação com outros algoritmos. Como resultado, é amplamente utilizado na filtragem de spam (identificar spam) e Análise de Sentimento (em análise de mídia social, para identificar sentimentos positivos e negativos dos clientes).
    - Previsões em tempo real: Naive Bayes é um voraz classificador de aprendizagem e com certeza rápido. Assim, pode ser usado para fazer previsões em tempo real.
    - Previsões multi-classes: Este algoritmo também é conhecido pela funcionalidade de previsão multi-classes. Aqui podemos prever a probabilidade de múltiplas classes das variáveis-alvo.
    - Sistema de Recomendação: o classificador e a filtragem colaborativa Naive Bayes em conjunto constroem um sistema de recomendação que utiliza técnicas de machine learning e mineração de dados para filtrar a informação invisível e prever se um usuário gostaria de um determinado recurso ou não.
- **Vantagens e condição bom desempenho:**
    - Quando a suposição de independência prevalece, um classificador Naive Bayes tem melhor desempenho em comparação com outros modelos como regressão logística, e você precisa de menos dados de treinamento.- É fácil e rápido para prever o conjunto de dados da classe de teste. Também tem um bom desempenho na previsão de classes múltiplas.
    - O desempenho é bom em caso de variáveis categóricas de entrada comparada com a variáveis numéricas. Para variáveis numéricas, assume-se a distribuição normal (curva de sino, que é uma suposição forte).
- **Desvantagens e condição de mau desempenho:**
    - Se a variável categórica tem uma categoria (no conjunto de dados de teste) que não foi observada no conjunto de dados 
	de treinamento, então o modelo irá atribuir uma probabilidade de 0 (zero) e não será capaz de fazer uma previsão. Isso 
	é muitas vezes conhecido como “Zero Frequency”. Para resolver isso, podemos usar a técnica de alisamento. Uma das técnicas
	mais simples de alisamento é a chamada estimativa de Laplace.
	- Por outro lado naive Bayes é também conhecido como um mau estimador, por isso, as probabilidades calculadas não devem 
	ser levadas muito a sério.
	- Outra limitação do Naive Bayes é a suposição de preditores independentes. Na vida real, é quase impossível que ter um 
	conjunto de indicadores que sejam completamente independentes.
- Pela consideração 3 na seção que relaciona as vantagens o desempenho pode ser comprometido pois 
    convertemos os atributos categóricos em numéricos. Além da condição 2 (Desvantagens) em que Naive Bayes é um mau estimador.
    Porém apesar de serem problemas não inviabiliza o uso no problema em questão.

fonte:https://www.vooo.pro/insights/6-passos-faceis-para-aprender-o-algoritmo-naive-bayes-com-o-codigo-em-python/

### Árvores de Decisão (Candidato)
- **Aplicações:**
    - Aprimorar a classificação de fragmentos de arquivos no processo de recuperação de dados em computação forense.
	Juliano Kazuki Matsuzaki Oya, Programa de Pós-graduação em Engenharia Elétrica, Brasília, Dezembro de 2016.
	- Aplicação da árvore de decisão para uma empresa agrícola do interior de São Paulo que cultiva laranja, auxiliando o 
	produtor de laranja a visualizar as principais decisões e seus respectivos resultados.Aplicação de árvore de decisão
	para a produção de laranja: uma proposição de plano na fazenda Recanto das Águas Sampaio, A.R.; Lima, E.H. de; Silva, 
	R.L.M. de; Nardi, P.C.C.
- **Vantagens e condição bom desempenho:**
	- Fácil de entender: saída da árvore de decisão é muito fácil de entender, mesmo para pessoas não analíticas. 
	Não requer nenhum conhecimento estatístico para lê-los e interpretá-los. Sua representação gráfica é muito intuitiva 
	e os usuários podem facilmente relacionar suas hipóteses.
	- Útil em Exploração de dados: a árvore de decisão é uma das formas mais rápidas de identificar as variáveis mais 
	significativas e a relação entre duas ou mais variáveis. Com a ajuda de árvores de decisão, podemos criar novas variáveis
	/ características que têm melhor poder para prever a variável alvo.
	- Menor necessidade de limpeza de dados: requer menos limpeza de dados em comparação com algumas outras técnicas de 
	modelagem. Não é influenciado por outliers e valores faltantes a um grau justo.
	- Tipo de dados não é uma restrição: Ela pode manipular variáveis numéricas e categóricas.
	- Método Não Paramétrico: a árvore de decisão é considerada um método não-paramétrico. Isto significa que as árvores de 
	decisão não têm hipóteses sobre a distribuição do espaço e a estrutura do classificador.
- **Desvantagens e condição de mau desempenho:**
	- Over fitting: over fitting é uma das dificuldades mais práticas para os modelos de árvores de decisão. Este problema 
	é resolvido através da definição de restrições sobre os parâmetros do modelo e poda.
	
	- Não adequado para variáveis contínuas: ao trabalhar com variáveis numéricas contínuas, a árvore de decisão perde 
	informações quando categoriza variáveis em diferentes categorias.
- As árvores de decisão pelas definições e consideraçãoes acima e traçando um paralelo com nosso modelo é um candidato. Porém existem variáveis contínuas e assim podendo haver perda de informação.

fonte:https://www.vooo.pro/insights/um-tutorial-completo-sobre-a-modelagem-baseada-em-tree-arvore-do-zero-em-r-python/

### AdaBoost (Candidato)
- **Aplicações:**
    - Na utilização de sistemas embarcados compostos por sensores para realização de análises e tomadas de decisão são cada vez 
    mais requisitados, principalmente onde se requer algum tipo de reconhecimento de padrão. 
    fonte:DocumentoDissertação de MestradoAutorChaves, Bruno Butilhão (Catálogo USP)
- **Vantagens e condição bom desempenho:**
	- Se o algoritmo de aprendizado utilizado garante
	aprendizagem fraca, então o AdaBoost
	retornará uma hipótese que classifica
	perfeitamente os dados de treinamento.
	- Aprendizagem fraca: melhor do que classificar
	os dados aleatoriamente.
	- Esse resultado é válido independentemente do
	espaço de hipóteses original e da complexidade
	da função que está sendo aprendida.
- **Desvantagens e condição de mau desempenho:**
	- Ao contrário de bagging, há o risco de
	super-ajuste (over-fitting).
		– Principalmente em casos onde há muito ruído
		na base de dados.
		– Caso haja erros de classificação nos dados
		de treinamento, boosting vai colocar muito
		peso nesses dados
- Como no caso do bagging pode ser combinado com outro algoritmo (ex. árvore de decisão) e
    não há restrição para uso no problema, considerando que não há ruído na base de dados.


### Configuração
Execute a célula de código abaixo para inicializar três funções de ajuda que você pode utilizar para treinar e testar os três modelos de aprendizagem supervisionada que você escolheu acima. As funções são as seguintes:
- `train_classifier` - recebe como parâmetro um classificador e dados de treinamento e ajusta o classificador aos dados.
- `predict_labels` - recebe como parâmetro um classificador ajustado, atributos e rótulo alvo e faz estimativas utilizando a pontuação do F<sub>1</sub>.
- `train_predict` - recebe como entrada um classificador, e dados de treinamento e teste, e executa `train_clasifier` e `predict_labels`.
 - Essa função vai dar a pontuação F<sub>1</sub> tanto para os dados de treinamento como para os de teste, separadamente.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Ajusta um classificador para os dados de treinamento. '''
    
    # Inicia o relógio, treina o classificador e, então, para o relógio
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Imprime os resultados
    print "O modelo foi treinado em {:.4f} segundos".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Faz uma estimativa utilizando um classificador ajustado baseado na pontuação F1. '''
    
    # Inicia o relógio, faz estimativas e, então, o relógio para
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Imprime os resultados de retorno
    print "As previsões foram feitas em {:.4f} segundos.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Treina e faz estimativas utilizando um classificador baseado na pontuação do F1. '''
    
    # Indica o tamanho do classificador e do conjunto de treinamento
    print "Treinando um {} com {} pontos de treinamento. . .".format(clf.__class__.__name__, len(X_train))
    
    # Treina o classificador
    train_classifier(clf, X_train, y_train)
    
    # Imprime os resultados das estimativas de ambos treinamento e teste
    print "Pontuação F1 para o conjunto de treino: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "Pontuação F1 para o conjunto de teste: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementação: Métricas de Desempenho do Modelo
Com as funções acima, você vai importar os três modelos de aprendizagem supervisionada de sua escolha e executar a função `train_prediction` para cada um deles. Lembre-se de que você vai precisar treinar e usar cada classificador para três diferentes tamanhos de conjuntos de treinamentos: 100, 200 e 300 pontos. Então você deve ter 9 saídas diferentes abaixo – 3 para cada modelo utilizando cada tamanho de conjunto de treinamento. Na célula de código a seguir, você deve implementar o seguinte:
- Importe os três modelos de aprendizagem supervisionada que você escolheu na seção anterior.
- Inicialize os três modelos e armazene eles em `clf_A`, `clf_B` e `clf_C`.
 - Defina um `random_state` para cada modelo, se a opção existir.
 - **Nota:** Utilize as configurações padrão para cada modelo – você vai calibrar um modelo específico em uma seção posterior.
- Crie diferentes tamanhos de conjuntos de treinamento para treinar cada modelo.
 - *Não embaralhe e distribua novamente os dados! Os novos pontos de treinamento devem ser tirados de `X_train` e `y_train`.*
- Treine cada modelo com cada tamanho de conjunto de treinamento e faça estimativas com o conjunto de teste (9 vezes no total).  
**Nota:** Três tabelas são fornecidas depois da célula de código a seguir, nas quais você deve anotar seus resultados.

In [7]:
# TODO: Importe os três modelos de aprendizagem supervisionada do sklearn
# from sklearn import model_A
from sklearn.naive_bayes import GaussianNB 
# from sklearn import model_B
from sklearn import tree
# from skearln import model_C
from sklearn.ensemble import AdaBoostClassifier

# TODO: Inicialize os três modelos
clf_A = GaussianNB()
clf_B = tree.DecisionTreeClassifier(random_state=0)
clf_C = AdaBoostClassifier(base_estimator=clf_B,random_state=0)

# TODO: Configure os tamanho dos conjuntos de treinamento
X_train_100, X_test, y_train_100, y_test = train_test_split(X_all, y_all, test_size=0.75, random_state=42)
X_train_200, X_test, y_train_200, y_test = train_test_split(X_all, y_all, test_size=0.50, random_state=42)
X_train_300, X_test, y_train_300, y_test = train_test_split(X_all, y_all, test_size=0.25, random_state=42)

# TODO: Executar a função 'train_predict' para cada classificador e cada tamanho de conjunto de treinamento
# train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
print "------------------------------------------------"
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
print "------------------------------------------------"
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)
print "================================================"
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
print "------------------------------------------------"
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
print "------------------------------------------------"
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)
print "================================================"
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
print "------------------------------------------------"
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
print "------------------------------------------------"
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Treinando um GaussianNB com 98 pontos de treinamento. . .
O modelo foi treinado em 0.0020 segundos
As previsões foram feitas em 0.0000 segundos.
Pontuação F1 para o conjunto de treino: 0.8065.
As previsões foram feitas em 0.0000 segundos.
Pontuação F1 para o conjunto de teste: 0.6870.
------------------------------------------------
Treinando um GaussianNB com 197 pontos de treinamento. . .
O modelo foi treinado em 0.0020 segundos
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de treino: 0.8118.
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de teste: 0.7273.
------------------------------------------------
Treinando um GaussianNB com 296 pontos de treinamento. . .
O modelo foi treinado em 0.0020 segundos
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de treino: 0.8058.
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de teste: 0.7591.
Treinando um DecisionTreeClassifier com 98

### Resultados em tabelas
Edite a célula abaixo e veja como a tabela pode ser desenhada em [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). Você deve salvar seus resultados abaixo nas tabelas fornecidas.

** Classificador 1 - ?**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |          0.0010            |       0.000                      |0.8065                            |0.6870           |
| 200                                |       0.0020        |       0.000                      |   0.8244                         | 0.7176     |
| 300                                | 0.0010                     | 0.0010                            | 0.8058                           |0.7591               |

** Classificador 2 - ?**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |      0.0010                |      0.0000                       |   1.0000                         | 0.6772                     |
| 200                                |    0.0020           |     0.0010                        |      1.0000                      |  0.7606                    |
| 300                                |  0.0020                    |    0.0000                         |  1.0000                          |0.6250               |

** Classificador 3 - ?**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                | 0.0040                    |   0.0010                          | 1.0000                           | 0.6929                     |
| 200                                | 0.0040                     | 0.0010                            |   1.0000                         | 0.7286                     |
| 300                                | 0.0040                     |0.0000                             | 1.0000                           |  0.6769                     |

## Escolhendo o Melhor Modelo
Nesta seção final, você irá escolher dos três modelos de aprendizagem supervisionada o *melhor* para utilizar os dados dos estudantes. Você então executará um busca em matriz otimizada para o modelo em todo o conjunto de treinamento (`X_train` e `y_train`) ao calibrar pelo menos um parâmetro, melhorando em comparação a pontuação F<sub>1</sub> do modelo não calibrado. 

### Questão 3 - Escolhendo o Melhor Modelo
*Baseando-se nos experimentos que você executou até agora, explique em um ou dois parágrafos ao conselho de supervisores qual modelo que você escolheu como o melhor. Qual modelo é o mais apropriado baseado nos dados disponíveis, recursos limitados, custo e desempenho?*

**Resposta: **
- Dos três modelos dois apresentaram f1_score máximo igual a 1 para o treinamento, estes foram a DecisionTree e adaBoost. Em relação ao desempenho (tempo de execução de treinamento e teste) o DecisionTree chegou a ser 4 vezes mais rápido no tempo de treinamento, porém se mantendo igual para os três testes no caso do tempo de teste. A pontuação f1_score para teste se manteve próxima tanto para a DecisionTree quanto para adaBoost (uma leve vantagem para o adaBoost). Sendo assim, considerando um custo de desenvolvimento igual para todos os modelos, o DecisionTree foi o escolhido pela questão do desempenho de tempo para o treinamento.

### Questão 4 – O Modelo para um Leigo
*Em um ou dois parágrafos, explique para o conselho de supervisores, utilizando termos leigos, como o modelo final escolhido deve trabalhar. Tenha certeza que você esteja descrevendo as melhores qualidades do modelo, por exemplo, como o modelo é treinado e como ele faz uma estimativa. Evite jargões técnicos ou matemáticos, como descrever equações ou discutir a implementação do algoritmo.*

**Resposta: **
- É uma abordagem que usa diagramas para mapear as várias alternativas e resultados de decisões, assim como as probabilidades de ocorrerem. A alternativa que proporciona o valor esperado mais alto é preferível. Essencialmente, árvores de decisões são diagramas que permitem representar e avaliar problemas que envolvem decisões sequenciais, colocando em destaque os riscos e os resultados identificados nos diversos cursos de ação. Para efectuar a representação gráfica da árvore de decisão são geralmente usadas linhas para identificar a decisão (por exemplo “sim” ou “não”) e nós para identificar a as questões sobre as quais se deve decidir. Cada um dos ramos formado por linhas e nós termina numa espécie de folha que identifica a consequência mais provável da sequência de decisões tomadas. fonte:wikipedia.
- Uma das vantagens da DecisionTree é a facilidade de se enxergar o problema e navegar pela árvore segue uma sequência lógica e intuitiva. A resposta procurada é encontrada no nó folha seguindo a lógica do topo para o final da árvore. Não faz suposições dos dados de treinamento pois faz uma abordagem não estatística e apresenta bons resultados para todos os tipos de dados do nosso problema e apresenta agilidade/reapidez de processamento para grandes quantidades de dados pois os dados vão sendo quebrados no decorrer do percurso da árvore. 

### Implementação: Calibrando o Modelo (_Tuning_)
Calibre o modelo escolhido. Utilize busca em matriz (`GridSearchCV`) com, pelo menos, um parâmetro importante calibrado com, pelo menos, 3 valores diferentes. Você vai precisar utilizar todo o conjunto de treinamento para isso. Na célula de código abaixo, você deve implementar o seguinte:
- Importe [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) e [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Crie o dicionário de parâmetros que você deseja calibrar para o modelo escolhido.
 - Examplo: `parameters = {'parameter' : [list of values]}`.
- Inicialize o classificador que você escolheu e armazene-o em `clf`.
- Crie a função de pontuação F<sub>1</sub> utilizando `make_scorer` e armazene-o em `f1_scorer`.
 - Estabeleça o parâmetro `pos_label` para o valor correto!
- Execute uma busca em matriz no classificador `clf` utilizando o `f1_scorer` como método de pontuação e armazene-o em `grid_obj`.
- Treine o objeto de busca em matriz com os dados de treinamento (`X_train`, `y_train`) e armazene-o em `grid_obj`.

In [8]:
# TODO: Importe 'GridSearchCV' e 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer
# TODO: Crie a lista de parâmetros que você gostaria de calibrar
parameters = {"max_depth": range(1,20), "random_state":[0]}

# TODO: Inicialize o classificador
clf = tree.DecisionTreeClassifier(random_state=0)
clf.fit(X_train,y_train)

# TODO: Faça uma função de pontuação f1 utilizando 'make_scorer' 
f1_scorer = make_scorer(f1_score,pos_label="yes",greater_is_better=True)

# TODO: Execute uma busca em matriz no classificador utilizando o f1_scorer como método de pontuação
grid_obj = GridSearchCV(estimator=clf,param_grid=parameters,scoring=f1_scorer)

# TODO: Ajuste o objeto de busca em matriz para o treinamento de dados e encontre os parâmetros ótimos
grid_obj = grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Reporte a pontuação final F1 para treinamento e teste depois de calibrar os parâmetrosprint "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "O modelo calibrado tem F1 de {:.4f} no conjunto de treinamento.".format(predict_labels(clf, X_train, y_train))
print "O modelo calibrado tem F1 de {:.4f} no conjunto de teste.".format(predict_labels(clf, X_test, y_test))

As previsões foram feitas em 0.0000 segundos.
O modelo calibrado tem F1 de 0.8354 no conjunto de treinamento.
As previsões foram feitas em 0.0000 segundos.
O modelo calibrado tem F1 de 0.8026 no conjunto de teste.


### Questão 5 - Pontuação F<sub>1</sub> Final
*Qual é a pontuação F<sub>1</sub> do modelo final para treinamento e teste? Como ele se compara ao modelo que não foi calibrado?*

**Resposta: **
F1 (treinamento) = 0.8351 inferior ao valor não calibrado e F1(teste) = 0.8026 superior ao valor não calibrado. Porém agora o tempo de execução da previsão para o treinamento foi 0.000 enquanto que no não calibrado foi 0.020.

> **Nota**: Uma vez que você completou todas as implementações de código e respondeu todas as questões acima com êxito, você pode finalizar seu trabalho exportando o iPython Nothebook como um document HTML. Você pode fazer isso utilizando o menu acima e navegando para  
**File -> Download as -> HTML (.html)**. Inclua a documentação final junto com o notebook para o envio do seu projeto.